In [1]:
import os
import csv
import sys
from pathlib import Path

sys.path.append(
    os.path.join(
        Path(os.getcwd()).parents[3],
        "src"
    )
)

csv.field_size_limit(sys.maxsize)

131072

In [2]:
import numpy as np
import pandas as pd
from models import AiNet
from representations import NGram
from logging import getLogger
from utils import cosine_distances, euclidean_distances

In [3]:
with open("cleansed_data.csv", "r+", newline="", encoding="utf-8") as f:
    reader = csv.reader(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data = list(reader)
    header = data.pop(0)

df = pd.DataFrame(data=data, columns=header)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134099 entries, 0 to 134098
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   id                       134099 non-null  object
 1   author                   134099 non-null  object
 2   statement                134099 non-null  object
 3   target                   134099 non-null  object
 4   BinaryNumTarget          134099 non-null  object
 5   manual_keywords          134099 non-null  object
 6   tweet                    134099 non-null  object
 7   5_label_majority_answer  134099 non-null  object
 8   3_label_majority_answer  134099 non-null  object
dtypes: object(9)
memory usage: 9.2+ MB


In [4]:
df["3_label_majority_answer"].value_counts()

Agree       128375
Disagree      5724
Name: 3_label_majority_answer, dtype: int64

In [5]:
df["5_label_majority_answer"].value_counts().sort_index()

Agree              54296
Disagree             561
Mostly Agree       53762
Mostly Disagree     2888
NO MAJORITY        22592
Name: 5_label_majority_answer, dtype: int64

In [8]:
sample = df.groupby(by=["5_label_majority_answer"], as_index=False, group_keys=False).apply(lambda c: c.sample(frac=0.05))
sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6705 entries, 1249 to 118041
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       6705 non-null   object
 1   author                   6705 non-null   object
 2   statement                6705 non-null   object
 3   target                   6705 non-null   object
 4   BinaryNumTarget          6705 non-null   object
 5   manual_keywords          6705 non-null   object
 6   tweet                    6705 non-null   object
 7   5_label_majority_answer  6705 non-null   object
 8   3_label_majority_answer  6705 non-null   object
dtypes: object(9)
memory usage: 523.8+ KB


In [9]:
sample.sample(2)

,id,author,statement,target,BinaryNumTarget,manual_keywords,tweet,5_label_majority_answer,3_label_majority_answer
70262,70311,W. Gardner,The U.S. Supreme Court decided in 1982 that no...,True,1,"Supreme Court, 1982, free k-12 education",@SheriB1957 @Tobymare @TomFitton Plyler vs. Do...,Mostly Agree,Agree
47873,47918,Tom Kertscher,The Pfizer COVID-19 vaccine is not approved.,False,0,"not approved,pfizer, vaccine",@lalalandlover51 @EastlickJason @PaulStanleyLi...,Mostly Agree,Agree


In [10]:
sample["3_label_majority_answer"].value_counts()

Agree       6397
Disagree     308
Name: 3_label_majority_answer, dtype: int64

In [11]:
sample["5_label_majority_answer"].value_counts().sort_index()

Agree              2715
Disagree             28
Mostly Agree       2688
Mostly Disagree     144
NO MAJORITY        1130
Name: 5_label_majority_answer, dtype: int64

In [12]:
np.round((df["3_label_majority_answer"].value_counts() / df.shape[0]) * 100, 2)

Agree       95.73
Disagree     4.27
Name: 3_label_majority_answer, dtype: float64

In [13]:
np.round((df["5_label_majority_answer"].value_counts() / df.shape[0]) * 100, 2)

Agree              40.49
Mostly Agree       40.09
NO MAJORITY        16.85
Mostly Disagree     2.15
Disagree            0.42
Name: 5_label_majority_answer, dtype: float64

In [10]:
np.round((sample["3_label_majority_answer"].value_counts() / sample.shape[0]) * 100, 2)

Agree       95.71
Disagree     4.29
Name: 3_label_majority_answer, dtype: float64

In [11]:
np.round((sample["5_label_majority_answer"].value_counts() / sample.shape[0]) * 100, 2)

Agree              40.49
Mostly Agree       40.09
NO MAJORITY        16.84
Mostly Disagree     2.15
Disagree            0.42
Name: 5_label_majority_answer, dtype: float64

In [15]:
representation = NGram()
features, result = representation.generate_representation(sample["tweet"].tolist())
result.shape

(13410, 30278)

In [16]:
sys.getsizeof(result)

1624112048

In [17]:
sys.getsizeof(result) / (2 ** 30)

1.5125722140073776

In [18]:
cosine_distances(result, result)

array([[0.        , 1.        , 0.9894057 , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 0.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.9894057 , 1.        , 0.        , ..., 1.        , 1.        ,
        0.98675853],
       ...,
       [1.        , 1.        , 1.        , ..., 0.        , 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 0.        ,
        1.        ],
       [1.        , 1.        , 0.98675853, ..., 1.        , 1.        ,
        0.        ]], dtype=float32)

In [19]:
root_logger = getLogger("root")

In [20]:
aiNet = AiNet(distance_method="cosine", logger=root_logger)

In [21]:
aiNet.fit(
    antigen_population=result, 
    max_iter=15, 
    number_of_antibodies=int(result.shape[0] * 0.10), 
    clone_multiplier=10, 
    no_best_cells_taken_each_selection=5, 
    percent_clones_reselected=0.12, 
    pruning_threshold=0.5
)

In [22]:
aiNet.antibody_population.shape

(9, 30278)

In [14]:
sample.to_csv("sample_cleansed_data.csv", encoding="utf-8", index=False, sep=",", quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

In [15]:
with open("sample_cleansed_data.csv", "r+", newline="", encoding="utf-8") as f:
    reader = csv.reader(f, delimiter=",", quoting=csv.QUOTE_MINIMAL, quotechar='"')
    data = list(reader)
    header = data.pop(0)

df1 = pd.DataFrame(data=data, columns=header)

In [21]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6705 entries, 0 to 6704
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       6705 non-null   object
 1   author                   6705 non-null   object
 2   statement                6705 non-null   object
 3   target                   6705 non-null   object
 4   BinaryNumTarget          6705 non-null   object
 5   manual_keywords          6705 non-null   object
 6   tweet                    6705 non-null   object
 7   5_label_majority_answer  6705 non-null   object
 8   3_label_majority_answer  6705 non-null   object
dtypes: object(9)
memory usage: 471.6+ KB


In [17]:
df1["3_label_majority_answer"].value_counts()

Agree       6397
Disagree     308
Name: 3_label_majority_answer, dtype: int64

In [18]:
df1["5_label_majority_answer"].value_counts()

Agree              2715
Mostly Agree       2688
NO MAJORITY        1130
Mostly Disagree     144
Disagree             28
Name: 5_label_majority_answer, dtype: int64

In [20]:
(df1["tweet"].isna().astype(int) + df1["tweet"].isnull().astype(int)).sum()

0